In [1]:
import tensorflow as tf
import numpy as np
import random

C:\Users\bin b zhu\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [39]:
class NeuralNetwork:
    def __init__(self, inodes, hnodes, onodes, learningrate):
        self.lr = learningrate
        
        wih = tf.Variable(tf.truncated_normal([inodes, hnodes], stddev=0.1))
        bih = tf.Variable(tf.constant(0.01, shape=[hnodes]))
        
        self.x = tf.placeholder(tf.float32, [None, inodes])
        hidden = tf.nn.relu(tf.matmul(self.x, wih) + bih)
        
        who = tf.Variable(tf.truncated_normal([hnodes, onodes], stddev=0.1))
        bho = tf.Variable(tf.constant(0.01, shape=[onodes]))
        
        self.y = tf.nn.softmax(tf.matmul(hidden, who) + bho)
        self.y_real = tf.placeholder(tf.float32, [None, onodes])
        
#         w = tf.Variable(tf.zeros([inodes, onodes]))
#         b = tf.Variable(tf.zeros([onodes]))
        
#         self.x = tf.placeholder(tf.float32, [None, inodes])
#         self.y = tf.nn.softmax(tf.matmul(self.x, w) + b)
#         self.y_real = tf.placeholder(tf.float32, [None, onodes])
        
        #self.cost = -tf.reduce_sum(self.y_real * tf.log(self.y))
        self.cost = tf.cost.cross_entropy(self.y, self.y_real)
        self.train_step = tf.train.GradientDescentOptimizer(self.lr).minimize(self.cost)
        
        evaluate = tf.equal(tf.argmax(self.y, axis=1), tf.argmax(self.y_real, axis=1))
        self.accuracy = tf.reduce_mean(tf.cast(evaluate, 'float'))
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        
    def train(self, inputs, targets):
        train_num = inputs.shape[0]
        for i in range(1000):
            batch_index = random.sample(range(train_num), 100)
            x_batch = inputs[batch_index]
            y_batch = targets[batch_index]
            
            _, cost = self.sess.run([self.train_step, self.cost], 
                                    feed_dict={self.x: x_batch, self.y_real: y_batch})
            
            if (i+1) % 100 == 0:
                print('round {} with cost: {}'.format(i+1, cost))
            
    def test(self, inputs, targets):
        accuracy = self.sess.run(self.accuracy, feed_dict={self.x: inputs, self.y_real: targets})
        print('accuracy: {}'.format(accuracy))
        
    def query(self, inputs):
        return self.sess.run(self.y, feed_dict={self.x: inputs})

In [40]:
inodes = 784
hnodes = 512
onodes = 10
learningrate = 0.1

n = NeuralNetwork(inodes, hnodes, onodes, learningrate)

AttributeError: module 'tensorflow._api.v1.losses' has no attribute 'cross_entropy'

In [6]:
def read_data(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    
    input_list = []
    target_list = []
    for record in lines:
        all_values = record.split(',')
        inputs = np.asfarray(all_values[1:]) / 255.0 * 0.99 + 0.01
        targets = np.zeros(10) + 0.01
        targets[int(all_values[0])] = 0.99
#         targets = np.zeros(10)
#         targets[int(all_values[0])] = 1

        input_list.append(inputs)
        target_list.append(targets)
    
    num = len(lines)    
    input_array = np.array(input_list).reshape(num, 784)
    target_array = np.array(target_list).reshape(num, 10)
    
    return input_array, target_array

In [7]:
train_inputs, train_targets = read_data('mnist_data/mnist_train.csv')

In [36]:
n.train(train_inputs, train_targets)

round 100 with cost: 0.12991535663604736
round 200 with cost: 0.11197242885828018
round 300 with cost: 0.11741175502538681
round 400 with cost: 0.10495136678218842
round 500 with cost: 0.09501896798610687
round 600 with cost: 0.08439755439758301
round 700 with cost: 0.10105239599943161
round 800 with cost: 0.0948493704199791
round 900 with cost: 0.09230821579694748
round 1000 with cost: 0.07722025364637375


In [9]:
test_inputs, test_targets = read_data('mnist_data/mnist_test.csv')

In [30]:
n.test(test_inputs, test_targets)

accuracy: 0.8970999717712402


In [34]:
test_inputs

array([[0.01, 0.01, 0.01, ..., 0.01, 0.01, 0.01],
       [0.01, 0.01, 0.01, ..., 0.01, 0.01, 0.01],
       [0.01, 0.01, 0.01, ..., 0.01, 0.01, 0.01],
       ...,
       [0.01, 0.01, 0.01, ..., 0.01, 0.01, 0.01],
       [0.01, 0.01, 0.01, ..., 0.01, 0.01, 0.01],
       [0.01, 0.01, 0.01, ..., 0.01, 0.01, 0.01]])

In [14]:
np.argmax(n.query(test_inputs[0].reshape(1, 784)))

7

In [15]:
np.argmax(test_targets[0])

7